# Data Preprocessing

In [2]:
import json
import pandas as pd
import os

Convert yelp dataset from json to csv 

In [3]:
# Define the path to the Yelp Dataset JSON files
data_path = '/Users/khushpatel/Desktop/Recommendation /Data Preprocessing/Json'

# Define the path where you want to save the CSV files
output_path = 'output_folder'

# Function to parse business JSON data and extract relevant fields
def parse_business_json(json_data):
    parsed_data = []
    for entry in json_data:
        business_id = entry['business_id']
        name = entry['name']
        address = entry['address']
        city = entry['city']
        state = entry['state']
        postal_code = entry['postal_code']
        latitude = entry['latitude']
        longitude = entry['longitude']
        stars = entry['stars']
        review_count = entry['review_count']
        is_open = entry['is_open']
        attributes = json.dumps(entry['attributes'])  # Convert attributes to JSON string
        categories = ','.join(entry['categories'])
        hours = json.dumps(entry.get('hours', {}))  # Convert hours to JSON string, handle missing values
        parsed_data.append([business_id, name, address, city, state, postal_code, latitude, longitude,
                            stars, review_count, is_open, attributes, categories, hours])
    return parsed_data

# Function to parse review JSON data and extract relevant fields
def parse_review_json(json_data):
    parsed_data = []
    for entry in json_data:
        review_id = entry['review_id']
        business_id = entry['business_id']
        user_id = entry['user_id']
        stars = entry['stars']
        date = entry['date']
        text = entry['text']
        parsed_data.append([review_id, business_id, user_id, stars, date, text])
    return parsed_data

# Create output folder if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Load and parse business data
business_parsed_data = []
with open(os.path.join(data_path, 'business.json'), 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        business_parsed_data.append(entry)
business_df = pd.DataFrame(business_parsed_data)
business_df.to_csv(os.path.join(output_path, 'business.csv'), index=False)

# Load and parse review data
review_parsed_data = []
with open(os.path.join(data_path, 'review.json'), 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        review_parsed_data.append(entry)
review_df = pd.DataFrame(review_parsed_data)
review_df.to_csv(os.path.join(output_path, 'review.csv'), index=False)

print("Conversion to CSV complete.")


Conversion to CSV complete.


From yelp business dataset we cept only those rows, which have **is_open = 1**

In [13]:
# Load the CSV file into a DataFrame
df = pd.read_csv('business.csv')

# Drop rows where the 'is_open' column value is 0
df = df[df['is_open'] != 0]

In [ ]:
df.head(10)

#### Calculating the Frequency of Data according to State.

In [14]:
# Get the frequency of each state
state_frequency = df['state'].value_counts()

print("State Frequencies:")
print(state_frequency)


State Frequencies:
state
PA     26289
FL     21540
TN      9600
IN      8946
MO      8363
AZ      8108
LA      7676
NJ      7031
NV      6277
AB      4346
CA      4065
ID      3783
DE      1894
IL      1765
TX         4
WA         2
MA         2
HI         1
UT         1
CO         1
MI         1
SD         1
XMS        1
VT         1
Name: count, dtype: int64


From all the states we kept row with state = **PA** 

In [18]:
new_df = df[df['state'] == 'PA']

new_df.head(100)
new_df.to_csv("business.csv")

Drop all coloumn except Business_id,name,stars,review_count,categories

In [20]:
required_columns = ["business_id", "name", "stars", "review_count", "categories"]
df = pd.read_csv("business.csv")
df = df[required_columns]

# Saving the modified DataFrame to a new CSV file
df.to_csv('business.csv', index=False)

In [10]:
# Load the business.csv and review.csv files into DataFrames
business_df = pd.read_csv('business.csv')
review_df = pd.read_csv('review.csv')

# Perform an inner join to keep only the reviews with business IDs present in the business.csv file
merged_df = pd.merge(review_df, business_df[['business_id','name','categories']], on='business_id', how='inner')

# Write the resulting DataFrame containing filtered reviews to a new CSV file
merged_df.to_csv('filtered_reviews.csv', index=False)

In [12]:
merged_df = pd.read_csv('filtered_reviews.csv')
merged_df.head(5)

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,date,name,categories
0,0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,2018-07-07 22:09:11,Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,2015-01-04 00:01:03,Zaika,"Halal, Pakistani, Restaurants, Indian"
2,2,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0,0.0,0.0,0.0,2013-06-24 11:21:25,Rittenhouse Grill,"Wine Bars, Restaurants, Nightlife, Steakhouses..."
3,3,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,0.0,0.0,0.0,2014-11-12 15:30:27,Good Karma Cafe,"Food, Cafes, Coffee & Tea, Restaurants"
4,4,J-4NdnDZ0pUQaUEEwDI9KQ,vrKkXsozqqecF3CW4cGaVQ,rjuWz_AD3WfXJc03AhIO_w,5.0,2.0,2.0,2.0,2012-12-04 16:46:20,The N Crowd,"Performing Arts, Arts & Entertainment, Nightli..."


In [14]:
df = pd.read_csv("filtered_reviews.csv")

# List of columns without the 'text' column
columns_without_text = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'date', 'name', 'categories']

# Dropping the 'text' column
merged_df = df[columns_without_text]

merged_df.to_csv("new.csv")


In [15]:
merged_df.head(10)

,review_id,user_id,business_id,stars,useful,funny,cool,date,name,categories
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,2018-07-07 22:09:11,Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,2015-01-04 00:01:03,Zaika,"Halal, Pakistani, Restaurants, Indian"
2,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0,0.0,0.0,0.0,2013-06-24 11:21:25,Rittenhouse Grill,"Wine Bars, Restaurants, Nightlife, Steakhouses..."
3,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,0.0,0.0,0.0,2014-11-12 15:30:27,Good Karma Cafe,"Food, Cafes, Coffee & Tea, Restaurants"
4,J-4NdnDZ0pUQaUEEwDI9KQ,vrKkXsozqqecF3CW4cGaVQ,rjuWz_AD3WfXJc03AhIO_w,5.0,2.0,2.0,2.0,2012-12-04 16:46:20,The N Crowd,"Performing Arts, Arts & Entertainment, Nightli..."
5,G_5UczbCBJriUAbxz3J7Tw,clWLI5OZP2ad25ugMVI8gg,x4XdNhp0Xn8lOivzc77J-g,5.0,0.0,0.0,0.0,2013-08-15 15:27:51,Thai Place Restaurant,"Thai, Restaurants"
6,DyrAIuKl60j_X8Yrrv-kpg,mNsVyC9tQVYtzLOCbh2Piw,MWmXGQ98KbRo3vsS5nZhMA,5.0,1.0,0.0,0.0,2014-10-27 02:47:28,Anthony's at Paxon Hollow,"Event Planning & Services, Italian, Venues & E..."
7,-P5E9BYUaK7s3PwBF5oAyg,Jha0USGDMefGFRLik_xFQg,bMratNjTG5ZFEA6hVyr-xQ,5.0,0.0,0.0,0.0,2017-02-19 13:32:05,Portobello Cafe,"Restaurants, Seafood, Cafes, Italian"
8,40thYphUgIfvJq17QCfTwA,QzCEzH3R7Z6erOGLr3t55Q,0pMj5xUAecW9o1P35B0AMw,5.0,1.0,0.0,1.0,2017-05-26 13:10:24,Wawa,"Food, Coffee & Tea, Gas Stations, Restaurants,..."
9,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5.0,0.0,0.0,0.0,2018-08-23 21:39:38,The Coventry Deli,"Restaurants, Delis, Salad, Sandwiches"


keeping only those business which have a category value as food/restaurant in categories column

In [17]:
df = pd.read_csv('new.csv')

# Drop rows with missing values in any column
df.dropna(inplace=True)

# Filter the DataFrame to keep only the rows where 'categories' contain either 'food' or 'restaurants'
filtered_df = df[df['categories'].str.contains('food|restaurants', case=False)]

filtered_df.to_csv("newFilteredReview.csv" , index=False)

filtered_df.head(5)


,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,date,name,categories
0,0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,2018-07-07 22:09:11,Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,2015-01-04 00:01:03,Zaika,"Halal, Pakistani, Restaurants, Indian"
2,2,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0,0.0,0.0,0.0,2013-06-24 11:21:25,Rittenhouse Grill,"Wine Bars, Restaurants, Nightlife, Steakhouses..."
3,3,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,0.0,0.0,0.0,2014-11-12 15:30:27,Good Karma Cafe,"Food, Cafes, Coffee & Tea, Restaurants"
5,5,G_5UczbCBJriUAbxz3J7Tw,clWLI5OZP2ad25ugMVI8gg,x4XdNhp0Xn8lOivzc77J-g,5.0,0.0,0.0,0.0,2013-08-15 15:27:51,Thai Place Restaurant,"Thai, Restaurants"


In [18]:
# Overview of Business/Restaurant Categories
merged_data = pd.read_csv("newFilteredReview.csv")

business_categories = merged_data.assign(categories = merged_data.categories.str.split(', ')).explode('categories')

print(str('The number of unique business categories is:'), len(business_categories))

# Most Common Business Categories

business_categories.categories.value_counts()

The number of unique business categories is: 5027979


categories
Restaurants        836680
Food               316944
Nightlife          241655
Bars               233431
American (New)     167425
                    ...  
Soccer                  5
Airport Lounges         5
Disc Golf               5
Framing                 5
Gardeners               5
Name: count, Length: 535, dtype: int64

In [19]:
filtered_categories2 = business_categories['categories'].value_counts()[business_categories['categories'].value_counts() > 5000]
# Display the filtered categories
#print(filtered_categories)


for index, value in filtered_categories2.items():
    print(f"{index}: {value}")

Restaurants: 836680
Food: 316944
Nightlife: 241655
Bars: 233431
American (New): 167425
American (Traditional): 165866
Breakfast & Brunch: 141660
Sandwiches: 134499
Italian: 110524
Pizza: 107659
Coffee & Tea: 85084
Seafood: 71515
Chinese: 65559
Event Planning & Services: 63197
Mexican: 58939
Specialty Food: 58853
Burgers: 57204
Desserts: 55080
Bakeries: 50155
Japanese: 49249
Wine & Spirits: 48011
Beer: 48011
Pubs: 47925
Cafes: 47666
Salad: 46949
Asian Fusion: 45316
Cocktail Bars: 43382
Sushi Bars: 42925
Cheesesteaks: 41587
Vegetarian: 39553
Fast Food: 37311
Steakhouses: 36330
Caterers: 33740
Gastropubs: 31704
Diners: 30359
Shopping: 29782
Mediterranean: 29723
Sports Bars: 29042
Ice Cream & Frozen Yogurt: 28839
Arts & Entertainment: 28508
Grocery: 28317
Delis: 27998
Local Flavor: 27724
Venues & Event Spaces: 26320
Juice Bars & Smoothies: 25879
Wine Bars: 25519
Vegan: 24976
Thai: 24795
Barbeque: 24179
Lounges: 23707
Ethnic Food: 21594
Beer Bar: 21534
Chicken Wings: 20145
Gluten-Free: 1806

merging the similar categories or the categories which occur together only

In [ ]:
df = pd.read_csv("newFilteredReview.csv")

def replace_categories(categories):
    categories = categories.replace('', 'Street Food')
    categories = categories.replace('Food Trucks', 'Street Food')
    return categories

# Applying the function to the 'categories' column
df['categories'] = df['categories'].apply(replace_categories)

# Displaying the updated DataFrame
df.to_csv("newFilteredReview.csv")

keeping only the most frequently occuring categories(i.e categories with frequency higher than 5000)

In [20]:
df = pd.read_csv("newFilteredReview.csv")

frequent_categories_list = ['Nightlife', 'Bars', 'American (New)', 'American (Traditional)', 'Sandwiches', 'Italian', 'Pizza', 
'Coffee & Tea', 'Seafood', 'Chinese', 'Mexican', 'Burgers', 'Desserts', 'Bakeries', 'Japanese', 'Salad', 'Asian Fusion', 'Sushi Bars', 
'Vegetarian', 'Fast Food', 'Steakhouses', 'Diners', 'Mediterranean', 'Ice Cream & Frozen Yogurt', 'Juice Bars & Smoothies', 'Vegan', 
'Thai', 'Barbeque', 'Lounges', 'Ethnic Food', 'Chicken Wings', 'Gluten-Free', 'Middle Eastern', 'Comfort Food', 'Bagels', 'Indian', 
'Noodles', 'Soup', 'Vietnamese', 'Korean', 'Tapas/Small Plates', 'French', 'Latin American', 'Food Stands', 'Greek', 'Tex-Mex', 'Donuts', 
'Caribbean', 'Halal', 'Bubble Tea', 'Szechuan', 'Ramen', 'Food Trucks', 'Southern', 'Tapas Bars', 'Tacos', 'Dance Clubs', 'Irish', 'Spanish']

# Step 1: Split the categories column into a list of categories
df['categories'] = df['categories'].apply(lambda x: x.split(', '))

# Step 2: Define a function to filter categories
def filter_categories(row):
    common_categories = list(set(row['categories']) & set(frequent_categories_list))
    if len(common_categories) > 0:
        return ','.join(common_categories[:3])  # Keep only the first three common categories
    else:
        return None  # Return None if no common categories found

# Step 3: Apply the filter_categories function to each row
df['categories'] = df.apply(filter_categories, axis=1)

# Step 4: Drop rows with None values in the categories column
df = df.dropna(subset=['categories'])

# Step 5: Optional - If you want to reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

df.to_csv("new_data.csv")

df.head(5)

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,date,name,categories
0,0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,2018-07-07 22:09:11,Turning Point of North Wales,"Coffee & Tea,American (New),Juice Bars & Smoot..."
1,1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,2015-01-04 00:01:03,Zaika,"Halal,Indian"
2,2,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0,0.0,0.0,0.0,2013-06-24 11:21:25,Rittenhouse Grill,"Nightlife,Bars,Seafood"
3,3,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,0.0,0.0,0.0,2014-11-12 15:30:27,Good Karma Cafe,Coffee & Tea
4,5,G_5UczbCBJriUAbxz3J7Tw,clWLI5OZP2ad25ugMVI8gg,x4XdNhp0Xn8lOivzc77J-g,5.0,0.0,0.0,0.0,2013-08-15 15:27:51,Thai Place Restaurant,Thai
